In [2]:
import cv2
import numpy
import os
import matplotlib.pyplot as plt
import mediapipe as mp
import mediapipe.python.solutions

import time

In [3]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [4]:
def mp_hands_detect(image, model): #in this case we pass in instance of Hands class object in model args
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Mediapipe accepts only RGB, so gotta convert native color BGR to RGB
    image.flags.writeable = False # saves memory, image no longer writeable for time being
    results = model.process(image) # image comes from OpenCV frame
    #print(results.multi_hand_landmarks)
    #print(type(results.multi_hand_landmarks))
    #print(len(results.multi_hand_landmarks) if type(results.multi_hand_landmarks) == list else print(0))
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # afterwards, we convert from RGB to BGR back
    return image, results

In [5]:
def render_hand_landmarks(image, results):
    if type(results.multi_hand_landmarks) == list:
        mp_drawing.draw_landmarks(image, results.multi_hand_landmarks[0], mp_hands.HAND_CONNECTIONS)


In [15]:
def style_hand_landmarks(image, results): # same as rendering hand land marks but with extra styling
    if type(results.multi_hand_landmarks) == list:
        mp_drawing.draw_landmarks(image, 
                                  results.multi_hand_landmarks[0], 
                                  mp_hands.HAND_CONNECTIONS,
                                  # arranged in BGR because remember we converted the image back from RGB to BGR
                                  mp_drawing.DrawingSpec(color=(86,255,255), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(170,86,255), thickness=2, circle_radius=4))

In [16]:
cap = cv2.VideoCapture(0)

# Begin use mediapipe
with mp_hands.Hands(max_num_hands = 1, min_detection_confidence=0.3, min_tracking_confidence=0.3) as hands:
    while cap.isOpened():
        ret, frame = cap.read()

        # Make detections
        image, results = mp_hands_detect(frame, hands)
        
        # Draw landmarks
        style_hand_landmarks(image, results)

        # Display to screen
        cv2.imshow('FSL Alphabet Detector Cam Test', image)
        # if q has been pressed for about 10 frames, quit
        if (cv2.waitKey(10) == ord('q')):
            break
    cap.release()
    cv2.destroyAllWindows()